<a href="https://colab.research.google.com/github/vee234o/promotion-model/blob/main/promotion_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score
import joblib
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_sample_weight
from ydata_profiling import ProfileReport
import joblib

In [39]:
#loaded the data
df = pd.read_csv('Promotion Dataset (1).csv')
df.head()

,EmployeeNo,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,YAK/S/00001,Commercial Sales and Marketing,MSc MBA and PhD,Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
1,YAK/S/00002,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
2,YAK/S/00003,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
3,YAK/S/00004,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
4,YAK/S/00006,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


In [40]:
# Generate a profiling reportProfileReport(df).to_file("report.html")
ProfileReport(df).to_file("report.html")

Summarize dataset:  58%|█████▊    | 14/24 [00:00<00:00, 22.27it/s, Describe variable: Past_Disciplinary_Action]

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 180.31it/s]


In [41]:
#dropped irrelevant columns
df1 = df.drop('EmployeeNo', axis = 1)

In [42]:
df1.isna().sum()

Division                                  0
Qualification                          1679
Gender                                    0
Channel_of_Recruitment                    0
Trainings_Attended                        0
Year_of_birth                             0
Last_performance_score                    0
Year_of_recruitment                       0
Targets_met                               0
Previous_Award                            0
Training_score_average                    0
State_Of_Origin                           0
Foreign_schooled                          0
Marital_Status                            0
Past_Disciplinary_Action                  0
Previous_IntraDepartmental_Movement       0
No_of_previous_employers                  0
Promoted_or_Not                           0
dtype: int64

In [43]:

# dropped missing values in promoted or not column and filled the missing values in the qualification column with its mode
df1 = df1.dropna(subset=['No_of_previous_employers', 'Promoted_or_Not'])
df1['Qualification'] = df1['Qualification'].fillna(df1['Qualification'].mode()[0])


In [44]:
df1.head()

,Division,Qualification,Gender,Channel_of_Recruitment,Trainings_Attended,Year_of_birth,Last_performance_score,Year_of_recruitment,Targets_met,Previous_Award,Training_score_average,State_Of_Origin,Foreign_schooled,Marital_Status,Past_Disciplinary_Action,Previous_IntraDepartmental_Movement,No_of_previous_employers,Promoted_or_Not
0,Commercial Sales and Marketing,MSc MBA and PhD,Female,Direct Internal process,2,1986,12.5,2011,1,0,41,ANAMBRA,No,Married,No,No,0,0
1,Customer Support and Field Operations,First Degree or HND,Male,Agency and others,2,1991,12.5,2015,0,0,52,ANAMBRA,Yes,Married,No,No,0,0
2,Commercial Sales and Marketing,First Degree or HND,Male,Direct Internal process,2,1987,7.5,2012,0,0,42,KATSINA,Yes,Married,No,No,0,0
3,Commercial Sales and Marketing,First Degree or HND,Male,Agency and others,3,1982,2.5,2009,0,0,42,NIGER,Yes,Single,No,No,1,0
4,Information and Strategy,First Degree or HND,Male,Direct Internal process,3,1990,7.5,2012,0,0,77,AKWA IBOM,Yes,Married,No,No,1,0


In [45]:
#split the data then saved my variables x and y
X = df1.drop('Promoted_or_Not', axis=1)
y = df1['Promoted_or_Not']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2, random_state = 42)


In [46]:
#encoded categorical columns
categorical_cols = X_train.select_dtypes(include=['object']).columns
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
#fit and trained them seperately to avoid data leakage
X_train[categorical_cols] = encoder.fit_transform(X_train[categorical_cols])
X_test[categorical_cols] = encoder.transform(X_test[categorical_cols])

In [49]:
#correlated the features and ranked them in descending order
df1.corr(numeric_only=True)['Promoted_or_Not'].sort_values(ascending=False)

Promoted_or_Not             1.000000
Targets_met                 0.224518
Previous_Award              0.201434
Training_score_average      0.178448
Last_performance_score      0.119690
Year_of_birth               0.017991
Year_of_recruitment         0.012287
No_of_previous_employers    0.001690
Trainings_Attended         -0.024345
Name: Promoted_or_Not, dtype: float64

In [53]:
#initialized the model
hgb = HistGradientBoostingClassifier(random_state=42, class_weight = 'balanced', categorical_features='from_dtype')


param_dist = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_iter': [100, 200, 300],        # equivalent to n_estimators
    'max_depth': [7, 9, 12],
    'l2_regularization': [0, 1.0]
}

#used randomized cv to search
random_search = RandomizedSearchCV(
    hgb,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    scoring='f1',
    n_jobs=-1,
    random_state=42
)

model = random_search.fit(X_train, y_train)


In [54]:
#to find out threshold to prevent bias
confidence_scores = random_search.predict_proba(X_test)[:, 1]

strictness_levels = np.arange(0.1, 1.0, 0.05)

print(f"{'Threshold':<10} | {'F1 Score':<10} | {'Recall':<10} | {'Precision':<10}")
print("-" * 50)

best_f1 = 0
best_thresh = 0

for current_rule in strictness_levels:
    decisions = (confidence_scores > current_rule).astype(int)
    
    # Calculate metrics for this specific boundary
    f1    = f1_score(y_test, decisions, zero_division=0)
    rec   = recall_score(y_test, decisions, zero_division=0)
    prec  = precision_score(y_test, decisions, zero_division=0)
    
    # Log results to visualize the trade-off
    print(f"{current_rule:.2f}       | {f1:.4f}     | {rec:.4f}     | {prec:.4f}")
    
    # Track the mathematically optimal F1 score
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = current_rule

print("-" * 50)
print(f"Optimal F1 Threshold Identified: {best_thresh:.2f}")

Threshold  | F1 Score   | Recall     | Precision 
--------------------------------------------------
0.10       | 0.3074     | 0.9955     | 0.1818
0.15       | 0.3201     | 0.9850     | 0.1911
0.20       | 0.3374     | 0.9760     | 0.2039
0.25       | 0.3467     | 0.9625     | 0.2115
0.30       | 0.3544     | 0.9505     | 0.2178
0.35       | 0.3603     | 0.9370     | 0.2231
0.40       | 0.3713     | 0.9295     | 0.2319
0.45       | 0.3752     | 0.9040     | 0.2367
0.50       | 0.3857     | 0.8726     | 0.2476
0.55       | 0.4000     | 0.8216     | 0.2644
0.60       | 0.4190     | 0.7211     | 0.2953
0.65       | 0.4408     | 0.6027     | 0.3475
0.70       | 0.4679     | 0.4813     | 0.4553
0.75       | 0.4844     | 0.4078     | 0.5965
0.80       | 0.4754     | 0.3553     | 0.7182
0.85       | 0.4672     | 0.3253     | 0.8282
0.90       | 0.4437     | 0.2924     | 0.9198
0.95       | 0.3832     | 0.2399     | 0.9524
--------------------------------------------------
Optimal F1 Threshold

In [76]:
FINAL_THRESHOLD = 0.65
probs = random_search.predict_proba(X_test)[:, 1]
final_preds = (probs > FINAL_THRESHOLD).astype(int)

In [77]:
print(classification_report(y_test, final_preds))

              precision    recall  f1-score   support

           0       0.96      0.89      0.92      6996
           1       0.35      0.60      0.44       667

    accuracy                           0.87      7663
   macro avg       0.65      0.75      0.68      7663
weighted avg       0.91      0.87      0.88      7663



In [78]:
ConfusionMatrixDisplay.from_predictions(y_test, final_preds, display_labels=["No", "Yes"], cmap='Blues')
plt.title(f"Promotion Prediction (Threshold {FINAL_THRESHOLD})")
plt.show()

In [79]:
gender_bias = df.groupby('Gender')['Promoted_or_Not'].mean()
print(gender_bias)


recruit_bias = df.groupby('Channel_of_Recruitment')['Promoted_or_Not'].mean()
print(recruit_bias)

state_bias = df.groupby('State_Of_Origin')['Promoted_or_Not'].mean().sort_values()
print(state_bias.head(3))
print(state_bias.tail(3))

Gender
Female    0.089048
Male      0.082701
Name: Promoted_or_Not, dtype: float64
Channel_of_Recruitment
Agency and others                  0.084280
Direct Internal process            0.083179
Referral and Special candidates    0.121287
Name: Promoted_or_Not, dtype: float64
State_Of_Origin
KWARA     0.067974
ABIA      0.068421
JIGAWA    0.068702
Name: Promoted_or_Not, dtype: float64
State_Of_Origin
TARABA    0.100000
GOMBE     0.103093
ONDO      0.117714
Name: Promoted_or_Not, dtype: float64


In [80]:
joblib.dump(model, 'promotion_model.pkl')

['promotion_model.pkl']